# Transform Orders Data - String to JSON
1. Pre-process the JSON String to fix the data quality issues
2. Transforms JSON String to JSON Object
3. Write transformed data to the Silver schema

In [ ]:
SELECT *
FROM udemyttstorage_catalog.bronze.orders_view;

1. Pre-process the JSON String to fix the data quality issues \
   [Documentation for regexp_replace function](https://learn.microsoft.com/en-us/azure/databricks/sql/language-manual/functions/regexp_replace)

In [ ]:
CREATE OR REPLACE TEMP VIEW orders_fixed_view
AS
SELECT
    regexp_replace('order_date": (\\d{4}-\\d{2}-\\d{2})', '"order_date": "\$1"') AS fixed_value
FROM udemyttstorage_catalog.bronze.orders_view;

2. Transforms JSON String to JSON Object \
   [Documentation of schema_of_json](https://learn.microsoft.com/en-us/azure/databricks/sql/language-manual/functions/schema_of_json) \
   [Documentation of from_json](https://learn.microsoft.com/en-us/azure/databricks/sql/language-manual/functions/from_json)

In [ ]:
# Get the schema
SELECT
    schema_of_json(fixed_value)
FROM orders_fixed_view
LIMIT 1;

In [ ]:
SELECT
    from_json(fixed_value, <schema from the above query>) AS json_value
FROM orders_fixed_view;

3. Write transformed data to the Silver schema

In [ ]:
CREATE TABLE IF NOT EXISTS udemyttstorage_catalog.silver.orders_json;
AS
SELECT
    from_json(fixed_value, <schema from the above query>) AS json_value
FROM orders_fixed_view;